In [1]:
import pandas as pd
import numpy as np

In [2]:
%%time
train_df = pd.read_csv('tap_fun_train.csv')
test_df = pd.read_csv('tap_fun_test.csv')

Wall time: 21.6 s


In [3]:
def cleaning(data):
    data['register_time'] = data.register_time.index
    return data

In [4]:
y = train_df.prediction_pay_price
train_df.drop(['prediction_pay_price'],axis=1,inplace = True)
train_df = cleaning(train_df)
test_df = cleaning(test_df)

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = scaler.fit_transform(train_df)
test_scale = scaler.transform(test_df)

In [6]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train_df, y, test_size = 0.1)

### Linear Regression

In [7]:
%%time
from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(x_train, y_train)
# preds = reg.predict(test_df)
print('Loss: {}'.format(reg.score(x_test,y_test)))

Loss: 0.4152026871068508
Wall time: 22 s


### Ridge Regression

In [8]:
%%time
from sklearn.linear_model import Ridge
reg2 = Ridge(alpha=3,normalize=True)
reg2.fit(x_train, y_train)
preds = reg2.predict(x_test)
# print('Loss: {}'.format(reg2.score(x_test,y_test)))

Wall time: 5 s


### Output Processing

In [9]:
## output processing
preds[preds<0.5] = 0
abb = pd.DataFrame(preds)
abb[(abb[0]>0) & (abb[0]<1)] = 0.99
sub = pd.DataFrame({'user_id':test_df.user_id,'prediction_pay_price': abb[0]},columns=['user_id','prediction_pay_price'])
sub.to_csv('output_processed_RF.csv',index=None)

In [10]:
sub.describe()

,user_id,prediction_pay_price
count,8.289340e+05,228801.000000
mean,1.811513e+06,2.166593
std,8.229500e+05,56.351862
min,1.493300e+04,0.000000
25%,1.286373e+06,0.000000
50%,2.012872e+06,0.000000
75%,2.481669e+06,0.000000
max,3.176543e+06,8628.335578


In [11]:
len(preds)

228801